# GitHub Report

## About this Notebook

<p>This Notebook provides a flexible reporting tool that creates a report based on either a users repositories and an organizations repository.  </p>
<p>For organizations with a large number of repositories, a Filter List can be used to limit the number of repos returned.</p>

This Notebook uses the following:
* Python 3 
* toc2 extension.

## Prerequisites

### GitHub Credentials File Setup


<div class="alert alert-block alert-info">
<b>Credential security file:</b> 
<p>In the same directory as this Notebook, create a file - gitsecrets.py - and add the following:<br>
    GITHUB_USER= Your Git User Name  <br>
    GITHUB_TOKEN= Your Git Token
</p>
</div>

<div class="alert alert-block alert-warning"><b>GitHub API Limits: </b>You may reach a limit of API calls that GitHub will allow, ex. 2000 calls/minute.  Try to limit the number of repos/folders/files. </div>


### Determine Reporting Strategy

<div class="alert alert-block alert-warning">
<b>Example config file:</b> Use yellow boxes for examples that are not inside code cells, or use for mathematical formulas if needed. Typically also used to display warning messages.
</div>


## Initialize Report

### Import packages and Fetch Repositories of interest

In [ ]:
import requests
import pprint
from gitsecrets import GITHUB_USER,GITHUB_TOKEN
# Create auth for Git credentials
auth=requests.auth.HTTPBasicAuth( GITHUB_USER  ,GITHUB_TOKEN )

userRepoUrl = 'https://api.github.com/user/repos'
orgRepoUrl = 'https://api.github.com/orgs/<Your Org>/repos'
#  Change myRepoUrl with on of the above as needed
myRepoUrl = userRepoUrl
repoResponse = requests.get(myRepoUrl, auth=auth)


# Get Repos List

## Org Repo List

In [ ]:
trOciRepoUrls = []
trOciRepoUrls.append('https://api.github.com/repos/tr/truccr_oci-iac')
trOciRepoUrls.append('https://api.github.com/repos/tr/truccr_oci-ca-dumptruck')
trOciRepoUrls.append('https://api.github.com/repos/tr/truccr_oci-manual-transmission')
trOciRepos =[]
for trOciRepoUrls in trOciRepoUrls:
    resp = repoResponse = requests.get(trOciRepoUrls, auth=auth)
    trOciRepos.append(resp.json())
print(len(trOciRepos))
repos = trOciRepos
pprint.pprint(trOciRepos)

## User Repo List

In [ ]:
repos =[]
for repoR in repoResponse.json():
    repos.append(repoR)
print('Number of Repos = ', str(len(repos)))


# Repos Report

In [ ]:
for repo in repos :
    print( 'Repo:',repo["name"])
    print('    Private:     ', repo['private'])
    print('    HTTPS URL:   ', repo['clone_url'])
    print('    SSH URL:     ', repo['ssh_url'])    
    contentsUrl = repo['contents_url']
    print('    Contents URL:' ,contentsUrl.replace('{+path}',''))
    contents = requests.get(contentsUrl.replace('{+path}',''),auth=auth)
    cFileList = []  ;   cDirList = []
    [cFileList.append(x) for x in contents.json() if x['type'] == 'file' ]
#- Files
    print('    Files:')
    for content in cFileList:
        print('       ', content['name'])
    [cDirList.append(x) for x in contents.json() if x['type'] == 'dir' ]
#- Folders
    print('    Folders:')
    for content in cDirList:
        print('       ', '/' + content['name'])

#- Branches
    print('    Branches:')
    branchs = requests.get(repo['branches_url'].replace('{/branch}',''),auth=auth)
    for branch in branchs.json():
        print('         ' ,branch['name'])
        branchContents = requests.get(repo['branches_url'].replace('{/branch}','/'+branch['name'])
                                      ,auth=auth)
        branchContent = branchContents.json()['commit']['commit']
        print('              Author:',branchContent['author'] ['name'])
        print('              Date:  ' ,branchContent ['author'] ['date'] )             
#-Folder Contents
    print('    Folder Contents:')
    for folder in cDirList:
        print('        /'+ folder['name'])
        fContents = requests.get(folder['url'] , auth=auth   )
        cFileList2 = []  ;   cDirList2 = []
        [cFileList2.append(x) for x in fContents.json() if x['type'] == 'file' ]
        for content in cFileList2:
            print('           '+ content['name'])
        [cDirList2.append(x) for x in fContents.json() if x['type'] == 'dir' ]            
        for content in cDirList2:
            print('           /'+ content['name'])     
            folder2 = folder['name'] + '/' + content['name']
#             print(contentsUrl.replace('{+path}',folder2 ))
            fContents2 = requests.get(contentsUrl.replace('{+path}',folder2),auth=auth)
            cFileList3 = []  ;   cDirList3 = []
            [cFileList3.append(x) for x in fContents2.json() if x['type'] == 'file' ]    
            for content2 in cFileList3:
                print('               ', content2['name'] )
            [cDirList3.append(x) for x in fContents2.json() if x['type'] == 'dir' ] 
            for content2 in cDirList3:
                print('               ', content2['name'] )                
#- Issues
    print('    Open Issues:')
    issues = requests.get(repo['issues_url'].replace('{/number}',''),auth=auth)
    for issue in issues.json():
        if 'open' in issue['state']:
            print('         ' ,str(issue['number']) + ' - ' +  issue['body'])    
    print('\n---' )
print('----------')
